## MULTI-LABEL IMAGE CLASSIFICATION WITH KERAS

### LOADING LIBRARIES

In [1]:
!pip install pillow
!pip install sklearn

    100% |████████████████████████████████| 2.0MB 674kB/s eta 0:00:01
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 12.4MB 111kB/s eta 0:00:01
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import PIL
import PIL.Image

from sklearn.utils import shuffle
import pandas as pd

In [26]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import numpy as np
#from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation

In [4]:

import glob,matplotlib as plt
%matplotlib inline

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [7]:
#from keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions

In [5]:
from keras.applications.xception import Xception,preprocess_input, decode_predictions

### LOADING DATA

In [6]:
train=pd.read_csv('./meta-data/train.csv')
test=pd.read_csv('./meta-data/test.csv')


In [7]:
img_width, img_height = 224, 224

In [8]:
train.shape,test.shape

((12600, 86), (5400, 1))

In [9]:

import cv2
def read_image(img_path,H,W):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (W,H)) # you can resize to  (128,128) or (256,256)
    return img


In [10]:
import gc
gc.collect()

11

In [11]:
%%time
image_lr=[]
train_img=[]
for i in list(train.Image_name):
    image_lr.append(read_image('./train_img/'+str(i),224,224))
train_img=np.array(image_lr,dtype=np.float32)
train_img=preprocess_input(train_img)

CPU times: user 3min 32s, sys: 4.03 s, total: 3min 36s
Wall time: 3min 36s


In [16]:
train_img.shape#,test_img.shape

(12600, 224, 224, 3)

In [13]:

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
#rescale = 1./255,   
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)

'''

test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)


'''


gc.collect()


0

In [14]:
Y=train.iloc[:,1:]
Y.shape

(12600, 85)

In [15]:
l=int(train.shape[0]-(0.1*train.shape[0]))
l
x_train=train_img[:l,:,:,:]
y_train=Y[:l]
x_valid=train_img[l:,:,:,:]
y_valid=Y[l:]
x_train.shape,y_train.shape,x_valid.shape,y_valid.shape


((11340, 224, 224, 3), (11340, 85), (1260, 224, 224, 3), (1260, 85))

In [16]:
batch_size=256

In [17]:


train_generator = train_datagen.flow(train_img,Y,
                                    batch_size = batch_size,
                                    shuffle=True)
                                    #class_mode = "categorical")
    
'''
validation_generator = test_datagen.flow(x_valid,y_valid,
                                         batch_size = batch_size,
                                        shuffle=True)
                                        #class_mode = "categorical")

'''

In [18]:
model = applications.Xception(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

83689472/83683744 [==============================] - 1s 0us/step: 


In [19]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers:
    layer.trainable = False

In [31]:


#Adding custom Layers 
x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(85, activation="sigmoid")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if sys.path[0] == '':


In [24]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 1

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [34]:
adam = Adam(lr=1e-2)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=[fbeta])

In [35]:
#validation_data=validation_generator, validation_steps=x_valid.shape[0]//batch_size, 
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=3,
                          shuffle=True)

Epoch 1/3
49/49 [==============================] - 162s 3s/step - loss: 0.4150 - fbeta: 0.7294 - val_loss: 0.2669 - val_fbeta: 0.8487
Epoch 2/3
49/49 [==============================] - 159s 3s/step - loss: 0.2471 - fbeta: 0.8493 - val_loss: 0.2060 - val_fbeta: 0.8783
Epoch 3/3
49/49 [==============================] - 159s 3s/step - loss: 0.2159 - fbeta: 0.8701 - val_loss: 0.1988 - val_fbeta: 0.8960


In [37]:
# Train the model 
K.set_value(adam.lr, 0.001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)
#model_final.fit(train_img, Y,batch_size=256,epochs=3,shuffle=True,verbose=1)

Epoch 1/2
49/49 [==============================] - 161s 3s/step - loss: 0.1879 - fbeta: 0.8883 - val_loss: 0.1699 - val_fbeta: 0.9062
Epoch 2/2
49/49 [==============================] - 158s 3s/step - loss: 0.1852 - fbeta: 0.8913 - val_loss: 0.1698 - val_fbeta: 0.9078


In [55]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model_final.layers[:110]:
    layer.trainable = False
for layer in model_final.layers[110:]:
    layer.trainable = True

In [56]:
adam = Adam(lr=1e-3)
model_final.compile(optimizer=adam, loss='binary_crossentropy', metrics=[fbeta])
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=1,
                          shuffle=True)

Epoch 1/1
49/49 [==============================] - 190s 4s/step - loss: 0.1762 - fbeta: 0.8976 - val_loss: 0.1499 - val_fbeta: 0.9174


In [57]:
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 188s 4s/step - loss: 0.1326 - fbeta: 0.9248 - val_loss: 0.1439 - val_fbeta: 0.9244
Epoch 2/2
49/49 [==============================] - 188s 4s/step - loss: 0.1137 - fbeta: 0.9366 - val_loss: 0.1311 - val_fbeta: 0.9279


In [66]:
K.set_value(adam.lr, 0.0001)
model_final.fit_generator( train_generator, steps_per_epoch=train_img.shape[0]//batch_size, epochs=2,
                          shuffle=True)

Epoch 1/2
49/49 [==============================] - 187s 4s/step - loss: 0.0953 - fbeta: 0.9468 - val_loss: 0.1027 - val_fbeta: 0.9441
Epoch 2/2
49/49 [==============================] - 185s 4s/step - loss: 0.0868 - fbeta: 0.9511 - val_loss: 0.0914 - val_fbeta: 0.9501


In [58]:
# serialize model to JSON
from keras.models import model_from_json
model_json = model_final.to_json()
with open("dl3-1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_final.save_weights("dl3-1.h5")
print("Saved model to disk")
 
# later...


Saved model to disk


### PREDICTIONG / SUBMISSION

##### LOAD MODEL AND WEIGHTS

In [40]:
gc.collect()

11630

In [39]:
%%time
image_lr=[]
test_img=[]
for i in list(test.Image_name):
    image_lr.append(read_image('./test_img/'+str(i),224,224))
test_img=np.array(image_lr,dtype=np.float32)
test_img=preprocess_input(test_img)

CPU times: user 1min 31s, sys: 1.38 s, total: 1min 32s
Wall time: 1min 31s


In [12]:
from keras.models import model_from_json
# load json and create model
json_file = open('dl3-1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_final = model_from_json(loaded_model_json)
# load weights into new model
model_final.load_weights("dl3-1.h5")
print("Loaded model from disk")



Loaded model from disk


In [15]:
test_datagen = ImageDataGenerator(
#rescale = 1./255,
horizontal_flip = True,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range=0.2,
rotation_range=20)

batch_size=256

In [68]:
#test_generator = test_datagen.flow(test_img,batch_size = batch_size,shuffle=True)                                      
#pred=model_final.predict_generator(test_generator,verbose=1)
pred=model_final.predict(test_img,batch_size,verbose=1)


5400/5400 [==============================] - 69s 13ms/step


In [21]:
test.shape,test_img.shape

((5400, 1), (5400, 224, 224, 3))

In [22]:
pred.shape

(5400, 85)

In [69]:
sam=pd.read_csv('./meta-data/sample_submission.csv')
sam['Image_name']=test.Image_name
sam.iloc[:,1:]=pred
sam.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,1.031856e-05,8.196082e-07,1.444669e-11,8.404958e-07,6.989886e-07,1.000000,5.067934e-07,1.525672e-06,9.999994e-01,...,1.803848e-05,4.970949e-08,5.830987e-06,1.000000,9.999998e-01,2.289729e-08,6.251568e-15,1.000000,4.774774e-07,1.000000
1,Image-2.jpg,3.859274e-02,1.683531e-01,3.854525e-05,5.453684e-01,7.770037e-03,0.982363,9.892535e-01,1.124911e-02,2.447692e-02,...,1.938736e-01,9.049323e-01,1.188473e-02,0.496242,2.778998e-02,1.835775e-03,2.441944e-02,0.201121,9.321152e-01,0.937876
2,Image-3.jpg,7.475208e-05,2.362567e-12,1.354042e-09,9.912577e-01,9.949375e-12,0.001659,9.999993e-01,1.262738e-06,2.000609e-06,...,9.969395e-01,2.197572e-04,9.235030e-05,0.993531,1.687343e-08,3.404775e-09,9.628648e-17,0.999336,1.177971e-06,1.000000
3,Image-4.jpg,9.980360e-01,9.867715e-05,1.000506e-01,9.525886e-01,4.838451e-04,0.782907,4.599632e-03,6.104806e-03,8.961533e-02,...,9.752880e-01,7.467472e-01,9.977804e-01,0.658075,6.259299e-03,2.788431e-01,1.800682e-11,0.955477,6.352916e-05,0.028090
4,Image-5.jpg,1.026996e-07,5.826743e-16,9.076603e-11,9.997582e-01,5.086981e-18,0.998061,1.178866e-04,3.078115e-11,3.938717e-07,...,5.284649e-07,1.205004e-13,4.613970e-08,1.000000,9.997498e-01,1.042505e-13,7.091864e-26,0.999998,8.108661e-13,0.000024


In [2]:
sam=pd.read_csv('sub-xception-4-prob.csv')

In [6]:
temp=sam.copy()
for i in sam.iloc[:,1:]:
    temp[i]=sam[i].apply(lambda x: 1 if x>0.4 else 0)
        

In [7]:
temp.head()

,Image_name,attrib_01,attrib_02,attrib_03,attrib_04,attrib_05,attrib_06,attrib_07,attrib_08,attrib_09,...,attrib_76,attrib_77,attrib_78,attrib_79,attrib_80,attrib_81,attrib_82,attrib_83,attrib_84,attrib_85
0,Image-1.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,1
1,Image-2.jpg,0,0,0,1,0,1,1,0,0,...,0,1,0,1,0,0,0,0,1,1
2,Image-3.jpg,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,1
3,Image-4.jpg,1,0,0,1,0,1,0,0,0,...,1,1,1,1,0,0,0,1,0,0
4,Image-5.jpg,0,0,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0


In [73]:
sam.to_csv('sub-xception-4-prob.csv',index=False)

In [8]:

temp.to_csv('sub-xception-5.csv',index=False)
temp.shape

(5400, 86)